In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# LeNet-5 model definition
class LeNet5(nn.Module):
    def __init__(self, num_classes=10, in_channels=3):
        super(LeNet5, self).__init__()
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Adjusting size based on input image size (32x32)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()

        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        # Forward pass through the network
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        # Flatten the tensor for fully connected layers
        x = x.view(-1, 16 * 5 * 5)  # Flattening 16 channels of 5x5 feature maps

        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Training and evaluation function (same as before)
def train_and_evaluate(model, train_loader, test_loader, num_epochs=20, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Loading and transforming datasets (same as before)
def load_data(dataset_name):
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
    if dataset_name == 'MNIST':
        train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
        test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
        in_channels = 1
    elif dataset_name == 'FMNIST':
        train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
        test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
        in_channels = 1
    elif dataset_name == 'CIFAR-10':
        train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
        test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
        in_channels = 3
    else:
        raise ValueError("Dataset not supported")

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    return train_loader, test_loader, in_channels

# Running training for each dataset
for dataset_name in ['MNIST', 'FMNIST', 'CIFAR-10']:
    print(f'\nTraining on {dataset_name}')
    train_loader, test_loader, in_channels = load_data(dataset_name)
    model = LeNet5(num_classes=10, in_channels=in_channels)
    train_and_evaluate(model, train_loader, test_loader)



Training on MNIST
Epoch [1/20], Loss: 0.3536
Epoch [2/20], Loss: 0.1087
Epoch [3/20], Loss: 0.0732
Epoch [4/20], Loss: 0.0562
Epoch [5/20], Loss: 0.0470
Epoch [6/20], Loss: 0.0375
Epoch [7/20], Loss: 0.0320
Epoch [8/20], Loss: 0.0286
Epoch [9/20], Loss: 0.0234
Epoch [10/20], Loss: 0.0215
Epoch [11/20], Loss: 0.0192
Epoch [12/20], Loss: 0.0155
Epoch [13/20], Loss: 0.0141
Epoch [14/20], Loss: 0.0134
Epoch [15/20], Loss: 0.0120
Epoch [16/20], Loss: 0.0116
Epoch [17/20], Loss: 0.0108
Epoch [18/20], Loss: 0.0079
Epoch [19/20], Loss: 0.0089
Epoch [20/20], Loss: 0.0077
Test Accuracy: 98.75%

Training on FMNIST
Epoch [1/20], Loss: 0.6872
Epoch [2/20], Loss: 0.4527
Epoch [3/20], Loss: 0.3832
Epoch [4/20], Loss: 0.3440
Epoch [5/20], Loss: 0.3163
Epoch [6/20], Loss: 0.2966
Epoch [7/20], Loss: 0.2812
Epoch [8/20], Loss: 0.2666
Epoch [9/20], Loss: 0.2552
Epoch [10/20], Loss: 0.2442
Epoch [11/20], Loss: 0.2365
Epoch [12/20], Loss: 0.2246
Epoch [13/20], Loss: 0.2179
Epoch [14/20], Loss: 0.2081
Epoch